In [ ]:
import time
from klampt import *
from klampt.math import vectorops,so3,se3
import math
from IPython.display import clear_output
import ipywidgets as widgets
from klampt.vis.ipython import KlamptWidget,EditPoint

world = WorldModel()
fn = "data/robots/ur5_with_gripper.rob"
plane = "data/terrains/block.off"
cylinder = "data/objects/cylinder.obj"
world.loadElement(plane)
world.loadElement(cylinder)
res = world.loadElement(fn)
robot = world.robot(0)
q = robot.getConfig()
q[2] = -math.pi/2
q[3] = math.pi/2
robot.setConfig(q)
cylinder = world.rigidObject(0)
cylinder.setTransform(so3.identity(),[0.5,0,0.5])
cylinder.appearance().setColor(1,0,0)
kvis = KlamptWidget(world,width=600,height=400)

#If you'd like to show the print output, comment out this line
#clear_output()
display(kvis)
controls = widgets.HBox([])
display(controls)

#Controls:
#left mouse click to rotate the view
#right click or ctrl+click to pan the view
#mouse wheel or shift+click to zoom the view

In [ ]:
from klampt.model import ik
import random

def lab3b(robot,qseed,ee_link,ee_local_position,ee_local_axis,target,target_axis):
    """
    TODO: complete me
    
    In:
    - robot: a RobotModel instance
    - qseed: a seed configuration of the robot.
    - ee_link: the index of the end effector's link
    - ee_local_position: the position of the end effector, expressed in the frame
      of link ee_link.
    - ee_local_axis: the rotational axis of the end effector, expressed in
      the frame of link ee_link.
    - target: a 3d target position (x,y,z)
    - target_axis: a 3d target direction for the end effector's axis (x,y,z)
    Out: a triple (q,errpos,erraxis) giving
    - q: the inverse kinematics optimized configuration of the robot, which (locally)
      minimizes the distance between the end effector position/axis and the target position/axis
    - errpos: The final distance between the end effector's world position
      and the target position. (should be as close as possible to 0 given a
      successful solution)
    - erraxis: The final distance between the end effector's world axis and
      the target axis.
    You will need to examine the documentation of the klampt.ik module, specifically
    the ik.objective and ik.solve functions.
    """
    #TODO: put your code here
    ##HINT: set up an IK objective as follows to strictly constrain the link's
    ##position and orientation
    #goal = ik.objective(robot.link(ee_link),R=desired_rotation_of_link,t=desired_translation_of_link)
    ##OR if you want to match some points lp1,lp2,..., given in the local
    ##coordinate frame to points wp1,wp2,..., given in the global coordinate
    ##frame
    #goal = ik.objective(robot.link(ee_link),local=[lp1,lp2,...],world=[wp1,wp2,...,])
    ##now seed the ik solver with the correct configuration (READ THE DOCS)...
    #???
    ##now solve
    #if ik.solve(goal):
    #    print "IK success!"
    ##   now the robot model's configuration is changed, and you need to
    ##   extract it. Your errpos and erraxis values should be very small
    #else:
    #    print "IK failure... returning best solution found"
    ##   the robot model's configuration is changed, but your errpos and
    ##   erraxis will not be close to 0

    h = 0.1
    link = robot.link(ee_link)
    goal = ik.objective(link,local=[ee_local_position,vectorops.madd(ee_local_position,ee_local_axis,h)],
                       world=[target,vectorops.madd(target,target_axis,h)])

    ee_world_position = ee_local_position
    ee_world_direction = ee_local_axis
    return (qseed,vectorops.distance(ee_world_position,target),vectorops.distance(ee_world_direction,target_axis))

def random_ik_seed(robot):
    """Returns an IK seed for use in the local IK solver"""
    q = [0.0]*robot.numLinks()
    qmin,qmax = robot.getJointLimits()
    for i in range(ee_link):
        q[i] = random.uniform(qmin[i],qmax[i])
    return q

def solve_ik(robot,ee_link,ee_local_position,ee_local_axis,target,target_axis):
    #the below line starts from the current configuration
    q = robot.getConfig()
    #the below line starts from the 0 configuration
    #q = [0.0]*robot.numLinks()
    #this performs a random restart
    qbest = q
#     dbest = float('inf')
#     for rr in range(10):
#         if rr > 0:
#             q = random_ik_seed(robot)
#         q,dp,da = lab3b(robot,q,ee_link,ee_local_position,ee_local_axis,target,target_axis)
#         if dp + da < 1e-2:
#             robot.setConfig(q)
#             if not robot.selfCollides():
#                 kvis.addText("HUDrr",x=1,y=40,text="Solved on restart "+str(rr))
#                 return (q,dp,da)
#         elif dp + da < dbest:
#             qbest = q[:]
#             dbest = dp+da
#     kvis.addText("HUDrr",x=1,y=40,text="Did not find solution")

    return lab3b(robot,qbest,ee_link,ee_local_position,ee_local_axis,target,target_axis)

In [ ]:
#run this cell to bind your code to the visualization playback window
ee_link = 7
ee_localpos = (0.15,0,0)
ee_localdir = (0,0,1)
target_position = cylinder.getTransform()[1]
target_axis = (0,0,1)
t = 0

kvis.addText("HUD1",(1,1))
kvis.addText("HUD2",(1,5))
kvis.addText("HUD3",(1,9))
kvis.addText("HUD4",(1,13))
kvis.addText("HUD5",(1,17))
kvis.addText("HUD6",(1,21))
kvis.addText("HUD7",(1,25))

def update_ik():
    global target_position,target_axis
    #aligns x to target_axis... we want z
    R = so3.canonical(list(target_axis))
    Rzx = so3.rotation([0,1,0],math.pi/2)
    world.rigidObject(0).setTransform(so3.mul(R,Rzx),target_position)

    q,dpos,daxis = solve_ik(robot,ee_link,ee_localpos,ee_localdir,target_position,target_axis)
    if not isinstance(q,list) or len(q) != robot.numLinks():
        print(q)
        raise ValueError("Improperly formatted return value")
    robot.setConfig(q)
    kvis.update()
    messages = []
    messages.append("Position error: %f, axis error %f"%(dpos,daxis))
    qmin,qmax = robot.getJointLimits()
    for i in range(min(len(q),7)):
        if q[i] < qmin[i] or q[i] > qmax[i]:
            messages.append("Joint %d position %f out of joint limits [%f,%f]"%(i,q[i],qmin[i],qmax[i]))
    if robot.selfCollides():
        messages.append("Robot self-collides")
    for (i,msg) in enumerate(messages):
        kvis.addText("HUD"+str(i+1),text=msg)
    for i in range(len(messages),6):
        kvis.addText("HUD"+str(i+1),text="")

def point_changed_cb(value):
    update_ik()

point_widget = EditPoint(target_position,[-1.0]*3,[1.0]*3,klampt_widget=kvis,point_radius=0.01,callback=point_changed_cb)

controls.children = (point_widget,)

### Written answers

TODO: Place written answers here